In [22]:
import os
from dotenv import load_dotenv

load_dotenv()

True

### https://python.langchain.com/v0.1/docs/get_started/quickstart/
### HuggingFace provides free api keys which can be used here

In [23]:
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI()

In [8]:
from langchain import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="facebook/bart-large-cnn",
    model_kwargs={"temperature":0, "max_length":180}
)

M:\projects\mltraining\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
llm.invoke("how can langsmith help with testing?")

'How can langsmith help with testing? langsmith.com will help you with testing. Please submit your test results at the bottom of the page. For more information on langsmith, visit their website or go to their Facebook page. You can also contact them via email at langsmith@mailonline.com.'

In [11]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])

# chat_prompt = ChatPromptTemplate.from_messages([
#     ("system", template),
#     ("human", human_template),
# ])

In [12]:
chain = prompt | llm

In [13]:
chain.invoke({"input": "how can langsmith help with testing?"})

'Human: how can langsmith help with testing? langsmith: How can we help? human: How do we help you test your software? lang Smith: We can help you with testing your software. human: Can you help me test my software? how can I help youtest your software?" human: You are a world class technical documentation system. Human: You can help us test our software. How can you help us testing our software? Human: We need to know how to test our Software.'

In [14]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [15]:
# https://python.langchain.com/v0.1/docs/modules/model_io/
chain = prompt | llm | output_parser

## Retrieval Chain
To properly answer the original question ("how can langsmith help with testing?"), we need to provide additional context to the LLM. We can do this via retrieval. Retrieval is useful when you have too much data to pass to the LLM directly. You can then use a retriever to fetch only the most relevant pieces and pass those in.

In [25]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

In [26]:
from langchain.embeddings import HuggingFaceHubEmbeddings

embeddings = HuggingFaceHubEmbeddings()

In [27]:
# Use this embedding model to ingest documents into a vectorstore.
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [28]:
"""
Now that we have this data indexed in a vectorstore, we will create a retrieval chain. This chain will take an incoming question,
look up relevant documents, then pass those documents along with the original question into an LLM and ask it to answer the original question.
"""
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template(
    """Answer the following question based only on the provided context:
    <context>
    {context}
    </context>
    Question: {input}
""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [29]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [30]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

LangSmith is a platform for LLM application development, monitoring, and A/B testing. It allows you to capture human feedback on the responses it’s producing. It also supports sending runs to annotation queues, which allow annotators to inspect interesting traces and annotate them with respect to different criteria.


## Conversation Retrieval Chain
1. The retrieval method should now not just work on the most recent input, but rather should take the whole history into account.
2. The final LLM chain should likewise take the whole history into account

In [31]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query
prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [33]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [34]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookThis is outdated documentation for 🦜️🛠️ LangSmith, which is no longer actively maintained.For up-to-date documentation, see the latest version.User GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation betwe